In [ ]:
"""WOrking on CM1 dataset"""

In [7]:
# Loading CM1 folder
import pandas as pd
import os

# Loading folder path
CM1_path = r"C:\Local Disk (A)\Github Files\Projects\Mass-Sceptra-Classification\Datasets\CM1"
data = []

# Combining all the files in the CM1 folder
for file in os.listdir(CM1_path):
    if file.endswith('.csv'):
        file_path = os.path.join(CM1_path, file)
        df = pd.read_csv(file_path, skiprows=1) # skipping the first row
        # Append to the list of DataFrames
        data.append(df)
# Concatenate all DataFrames into a single DataFrame
combined_CM1_data = pd.concat(data, ignore_index=True)

print(combined_CM1_data.head(5))
print("Rows & columns:", combined_CM1_data.shape)


   #Point  X(Thompsons)    Y(Counts)
0       0          40.0    46.235340
1       1          40.9    58.811787
2       2          41.0   617.287781
3       3          41.2    62.857056
4       4          42.1  3057.952637
Rows & columns: (92310, 3)


In [8]:
# Preprocessing Data

# summary of the datset
print("Summary :")
summary = combined_CM1_data.describe()
print(summary)

# Cheking for missing values
print("")
print("Cheking for missing values :")
is_null = combined_CM1_data.isnull().sum()
print(is_null)

# dropping "#Point" column as it is not relevant
print("")
print("After droping '#Point' column :")
combined_CM1_data = combined_CM1_data.drop('#Point', axis= 1)
print(combined_CM1_data.head(5))

# Rounding up the values of X(Thompsons) column
print("")
print("After Rounding :")
import numpy as np
combined_CM1_data['X(Thompsons)'] = np.ceil(combined_CM1_data['X(Thompsons)'])
print(combined_CM1_data.head(5))

# Normalizing the dataset
"""print("")
print("After Normalization :")
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

scaler = MinMaxScaler()
columns_to_normalize = ['X(Thompsons)', 'Y(Counts)']
combined_CM1_data[columns_to_normalize] = scaler.fit_transform(combined_CM1_data[columns_to_normalize])
print(combined_CM1_data.head(5))"""

Summary :
             #Point  X(Thompsons)      Y(Counts)
count  92310.000000  92310.000000   92310.000000
mean     171.147308    143.243818    1193.070302
std      107.140743     68.833009    9722.745794
min        0.000000     39.900000       0.003136
25%       82.000000     88.200000      22.855255
50%      164.000000    134.000000      70.941631
75%      248.000000    186.100000     283.324089
max      521.000000    413.200000  654898.562500

Cheking for missing values :
#Point          0
X(Thompsons)    0
Y(Counts)       0
dtype: int64

After droping '#Point' column :
   X(Thompsons)    Y(Counts)
0          40.0    46.235340
1          40.9    58.811787
2          41.0   617.287781
3          41.2    62.857056
4          42.1  3057.952637

After Rounding :
   X(Thompsons)    Y(Counts)
0          40.0    46.235340
1          41.0    58.811787
2          41.0   617.287781
3          42.0    62.857056
4          43.0  3057.952637


'print("")\nprint("After Normalization :")\nfrom sklearn.preprocessing import MinMaxScaler\nimport pandas as pd\n\nscaler = MinMaxScaler()\ncolumns_to_normalize = [\'X(Thompsons)\', \'Y(Counts)\']\ncombined_CM1_data[columns_to_normalize] = scaler.fit_transform(combined_CM1_data[columns_to_normalize])\nprint(combined_CM1_data.head(5))'

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Split dataset into features and target
X = combined_CM1_data[['X(Thompsons)']]  # Feature: X(Thompsons)
y = combined_CM1_data['Y(Counts)']  # Target: Y(Counts)

# Split data into 70% training and 30% testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=50)

# Initialize LightGBM Regressor
model = lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=31, learning_rate=0.05, n_estimators=100)#
#model = lgb.LGBMRegressor(boosting_type='gbdt', num_leaves=31, learning_rate=0.05, n_estimators=100)

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model using Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse:.2f}")

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000131 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 255
[LightGBM] [Info] Number of data points in the train set: 64617, number of used features: 1
[LightGBM] [Info] Start training from score 1194.049982
Mean Squared Error: 95602970.90


In [10]:
# Convert Y(Counts) into categories
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

bins = [0, 100, 1000, float('inf')]  # Define ranges for bins (adjust as needed)
labels = ['Low', 'Medium', 'High']  # Define labels for bins
combined_CM1_data['Y(Counts)_category'] = pd.cut(combined_CM1_data['Y(Counts)'], bins=bins, labels=labels)

# Use the new categorical target for classification
X = combined_CM1_data[['X(Thompsons)']]  # Feature: X(Thompsons)
y = combined_CM1_data['Y(Counts)_category']  # Target: Y(Counts) as categories

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=50)

# Initialize LightGBM Classifier
model = lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=31, learning_rate=0.05, n_estimators=100)

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model using Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000112 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 255
[LightGBM] [Info] Number of data points in the train set: 64617, number of used features: 1
[LightGBM] [Info] Start training from score -2.039175
[LightGBM] [Info] Start training from score -0.557073
[LightGBM] [Info] Start training from score -1.214088


NameError: name 'accuracy_score' is not defined

In [ ]:
"""WOrking on CM2 dataset"""

In [ ]:
"""WOrking on CM3 dataset"""

In [ ]:
"""WOrking on CM1,CM2,CM3 combined"""